In [21]:
#Import libraries:
import pandas as pd
import numpy as np
import missingno as mn
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.metrics import confusion_matrix,roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, classification_report


import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [22]:
#Normalizing function for columns
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        if feature_name!='application_key':# or feature_name!= "application_key" or feature_name!= "default_ind":
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [23]:
#Selecting specified number of columns with a certain NaN values

def get_column(df):
    feature_NA = []
    for feature_name in df.columns:
        if df[feature_name].isna().sum()>0.60*df.shape[0]:
            feature_NA.append(feature_name)
    return feature_NA

#Dropping Columns
def drop_column(df,feature_NA):
    df = df.drop(columns=feature_NA,axis=1)
    return df
    

In [24]:
def data_preprocess_spliting(dataset):
    #Converting dataset
    dataset = dataset.replace("L", 1, regex=True)
    dataset = dataset.replace("C", 0, regex=True)
    dataset = dataset.replace("na", np.nan, regex=True)
    dataset = dataset.replace("missing", np.nan, regex=True)
    dataset=dataset.astype('float64')
    
    #Spliting C and L from dataset
    dataC,dataL = [x for _, x in dataset.groupby(dataset['mvar47'] == 1.0)]
    return dataC,dataL

In [25]:
def data_preprocess_NA(dataC,dataL):
    
    #Removing Columns
    
    dataC= drop_column(dataC,featureC_NA_global)
    dataL= drop_column(dataL,featureL_NA_global)
    
    #Removing rows containing high percentage of NaN values
    
    #dataC = dataC.dropna(axis=0,thresh=0.75*dataC.shape[1])
    #dataL = dataL.dropna(axis=0,thresh=0.75*dataL.shape[1])
    
    #Normalizing data
    dataC= normalize(dataC)
    dataL= normalize(dataL)
    
    #imputing data
    dataC = dataC.fillna(dataC.median())
    dataL = dataL.fillna(dataL.median())
    
    return dataC,dataL

In [26]:
dataset = pd.read_csv("Training_dataset_Original.csv", low_memory=False)
dataset_lead = pd.read_csv("Leaderboard_dataset.csv", low_memory=False)

In [27]:
dataC, dataL = data_preprocess_spliting(dataset)

featureC_NA_global = get_column(dataC)
featureL_NA_global = get_column(dataL)

dataC, dataL = data_preprocess_NA(dataC,dataL)

In [28]:
dataC_lead, dataL_lead = data_preprocess_spliting(dataset_lead)
dataC_lead, dataL_lead = data_preprocess_NA(dataC_lead,dataL_lead)

In [41]:
print dataL.shape, dataC.shape, dataL_lead.shape, dataC_lead.shape

(28160, 45) (51840, 45) (8818, 44) (16182, 44)


In [30]:
X_dataL_train = dataL.values[:,1:dataL.shape[1]-2]
Y_dataL_train = dataL.values[:,dataL.shape[1]-1]
X_dataC_train = dataC.values[:,1:dataC.shape[1]-2]
Y_dataC_train = dataC.values[:,dataC.shape[1]-1]

In [31]:
X_keyL_lead = dataL_lead.values[:,0]
X_keyC_lead = dataC_lead.values[:,0]
X_dataL_test = dataL_lead.values[:,1:dataL_lead.shape[1]-1]
X_dataC_test = dataC_lead.values[:,1:dataC_lead.shape[1]-1]

In [32]:
#from sklearn.cross_validation import train_test_split
#X_dataL_train, X_dataL_test, Y_dataL_train, Y_dataL_test = train_test_split( X_dataL, Y_dataL, test_size = 0.3, random_state = 18)
#X_dataC_train, X_dataC_test, Y_dataC_train, Y_dataC_test = train_test_split( X_dataC, Y_dataC, test_size = 0.3, random_state = 18)

Y_dataL_test = Y_dataL_key2[:,1]
Y_dataC_test = Y_dataC_key2[:,1]
Y_dataL_train = Y_dataL_key1[:,1]
Y_dataC_train= Y_dataC_key1[:,1]

def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=2, early_stopping_rounds=5):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=2,
            metrics='auc', early_stopping_rounds=5)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['application_key'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

#Choose all predictors except target & IDcols
predictors = [x for x in dataL.columns if x not in [target, IDcol]]

xgb1 = XGBClassifier(
 learning_rate =0.2,
 n_estimators=100,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, dataL, predictors)

#Choose all predictors except target & IDcols
predictors = [x for x in dataC.columns if x not in [target, IDcol]]

xgb2 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, dataC, predictors)

In [33]:
X_train = X_dataL_train
y_train = Y_dataL_train
X_test = X_dataL_test
#y_test = Y_dataC_test


#XBoost on L

# fit model no training data

params = {'silent':False,
        'scale_pos_weight':1,
        'learning_rate':0.1,
        'colsample_bytree' : 0.8,
        'subsample' : 0.8,
        'objective' :'binary:logistic', 
        'n_estimators':1000, 
        'reg_alpha': 0.3,
        'max_depth':6, 
        'gamma':1,
        'booster':'gbtree',
        'min_child_weight':4,
        'eta': 0.025,
        'seed':0,
        }

#eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ["auc","error"]

xgtrain = xgb.DMatrix(X_train, y_train)

model_L = XGBClassifier(**params)

#do cross validation
print ('Start cross validation')
cvresult_L = xgb.cv(params, xgtrain, num_boost_round=5000, nfold=10, metrics=['auc'],
     early_stopping_rounds=50, stratified=True, seed=1301)

print('Best number of trees = {}'.format(cvresult_L.shape[0]))
model_L.set_params(n_estimators=cvresult_L.shape[0])

%time model_L.fit(X_train, y_train, eval_metric=eval_metric, verbose=True)

print('Overall AUC:', roc_auc_score(y_train, model_L.predict_proba(X_train)[:,1]))
print('Predict the probabilities based on features in the test set')
pred = model_L.predict_proba(X_test, ntree_limit=cvresult_L.shape[0])




#model_C.fit(X_dataC, Y_dataC)
# make predictions for test data
y_prob_L = model_L.predict_proba(X_test)
y_pred_L = model_L.predict(X_test)



Start cross validation
[17:28:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[17:28:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[17:28:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=6
[17:28:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 2 pruned nodes, max_depth=6
[17:28:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 4 pruned nodes, max_depth=6
[17:28:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 6 pruned nodes, max_depth=6
[17:28:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 2 pruned nodes, max_depth=6
[17:28:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 2 pruned nodes, max_depth=6
[17:28:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra 

[17:28:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=6
[17:28:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 2 pruned nodes, max_depth=6
[17:28:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 4 pruned nodes, max_depth=6
[17:28:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 2 pruned nodes, max_depth=6
[17:28:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 4 pruned nodes, max_depth=6
[17:28:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 2 pruned nodes, max_depth=6
[17:28:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 2 pruned nodes, max_depth=6
[17:28:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 10 pruned nodes, max_depth=6
[17:28:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 2 pruned node

[17:28:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[17:28:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 8 pruned nodes, max_depth=6
[17:28:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 4 pruned nodes, max_depth=6
[17:28:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 2 pruned nodes, max_depth=6
[17:28:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 2 pruned nodes, max_depth=6
[17:28:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 4 pruned nodes, max_depth=6
[17:28:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 6 pruned nodes, max_depth=6
[17:28:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 2 pruned nodes, max_depth=6
[17:28:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 6 pruned nodes, max_

[17:28:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 0 pruned nodes, max_depth=6
[17:28:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 2 pruned nodes, max_depth=6
[17:28:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 4 pruned nodes, max_depth=6
[17:28:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 4 pruned nodes, max_depth=6
[17:28:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 6 pruned nodes, max_depth=6
[17:28:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 2 pruned nodes, max_depth=6
[17:28:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 10 pruned nodes, max_depth=6
[17:28:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 4 pruned nodes, max_depth=6
[17:28:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 2 pruned nodes, max

[17:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 2 pruned nodes, max_depth=6
[17:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 2 pruned nodes, max_depth=6
[17:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 4 pruned nodes, max_depth=6
[17:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 2 pruned nodes, max_depth=6
[17:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 0 pruned nodes, max_depth=6
[17:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 6 pruned nodes, max_depth=6
[17:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 2 pruned nodes, max_depth=6
[17:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 4 pruned nodes, max_depth=6
[17:28:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 6 pruned nodes, max_

...

[17:29:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[17:29:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 4 pruned nodes, max_depth=6
[17:29:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=6
[17:29:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 2 pruned nodes, max_depth=6
[17:29:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 4 pruned nodes, max_depth=6
[17:29:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=6
[17:29:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=6
[17:29:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 2 pruned nodes, max_depth=6
[17:29:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_

[17:29:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 0 pruned nodes, max_depth=6
[17:29:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 2 pruned nodes, max_depth=6
[17:29:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 4 pruned nodes, max_depth=6
[17:29:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 4 pruned nodes, max_depth=6
[17:29:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=6
[17:29:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 2 pruned nodes, max_depth=6
[17:29:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 2 pruned nodes, max_depth=6
[17:29:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 4 pruned nodes, max_depth=6
[17:29:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 2 pruned nodes, max_

[17:29:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 4 pruned nodes, max_depth=6
[17:29:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 2 pruned nodes, max_depth=6
[17:29:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 0 pruned nodes, max_depth=6
[17:29:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 4 pruned nodes, max_depth=6
[17:29:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 0 pruned nodes, max_depth=6
[17:29:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=6
[17:29:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=6
[17:29:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 2 pruned nodes, max_depth=6
[17:29:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 2 pruned nodes, max_

[17:29:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 2 pruned nodes, max_depth=6
[17:29:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=6
[17:29:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 8 pruned nodes, max_depth=6
[17:29:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 10 pruned nodes, max_depth=6
[17:29:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 2 pruned nodes, max_depth=6
[17:29:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 4 pruned nodes, max_depth=6
[17:29:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 6 pruned nodes, max_depth=6
[17:29:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 2 pruned nodes, max_depth=6
[17:29:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 4 pruned nodes, max

/home/shahil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



predictions = [round(value) for value in y_pred_L]
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred_L)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [34]:
#Event_Probability_L and DataFrame  creation
event_prob_L = np.amax(y_prob_L, axis=1)
d_L= {"application_key":X_keyL_lead.astype(int), "y_pred" : y_pred_L,"event_prob" : event_prob_L}
df_L = pd.DataFrame(data=d_L)


In [35]:
X_train = X_dataC_train
y_train = Y_dataC_train
X_test = X_dataC_test
#y_test = Y_dataC_test


#XBoost on C

# fit model no training data

params = {'silent':False,
        'scale_pos_weight':1,
        'learning_rate':0.1,
        'colsample_bytree' : 0.8,
        'subsample' : 0.8,
        'objective' :'binary:logistic', 
        'n_estimators':1000, 
        'reg_alpha': 0.3,
        'max_depth':6, 
        'gamma':0.1,
        'booster':'gbtree',
        'min_child_weight':4,
        'eta': 0.025,
        'seed':0,
        }

#eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ["auc","error"]

xgtrain = xgb.DMatrix(X_train, y_train)

model_C = XGBClassifier(**params)

#do cross validation
print ('Start cross validation')
cvresult_C = xgb.cv(params, xgtrain, num_boost_round=5000, nfold=15, metrics=['auc'],
     early_stopping_rounds=50, stratified=True, seed=1301)

print('Best number of trees = {}'.format(cvresult_C.shape[0]))
model_C.set_params(n_estimators=cvresult_C.shape[0])

%time model_C.fit(X_train, y_train, eval_metric=eval_metric, verbose=True)

print('Overall AUC:', roc_auc_score(y_train, model_C.predict_proba(X_train)[:,1]))
print('Predict the probabilities based on features in the test set')
pred = model_C.predict_proba(X_test, ntree_limit=cvresult_C.shape[0])




#model_C.fit(X_dataC, Y_dataC)
# make predictions for test data
y_prob_C = model_C.predict_proba(X_test)
y_pred_C = model_C.predict(X_test)





Start cross validation
[17:29:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[17:29:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[17:29:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[17:29:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[17:29:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[17:29:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[17:29:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[17:29:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[17:29:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 e

[17:30:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 2 pruned nodes, max_depth=6
[17:30:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[17:30:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[17:30:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[17:30:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[17:30:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 2 pruned nodes, max_depth=6
[17:30:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[17:30:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[17:30:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned no

[17:30:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[17:30:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[17:30:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[17:30:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 2 pruned nodes, max_depth=6
[17:30:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[17:30:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[17:30:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 2 pruned nodes, max_depth=6
[17:30:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 2 pruned nodes, max_depth=6
[17:30:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nod

[17:30:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[17:30:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[17:30:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 4 pruned nodes, max_depth=6
[17:30:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[17:30:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=6
[17:30:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[17:30:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[17:30:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[17:30:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned node

[17:30:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[17:30:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[17:30:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 2 pruned nodes, max_depth=6
[17:30:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=6
[17:30:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=6
[17:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[17:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[17:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[17:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes,

...

[17:34:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=6
[17:34:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=6
[17:34:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=6
[17:34:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 0 pruned nodes, max_depth=6
[17:34:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=6
[17:34:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[17:34:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=6
[17:34:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=6
[17:34:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max

[17:34:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[17:34:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 0 pruned nodes, max_depth=6
[17:34:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=6
[17:34:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 2 pruned nodes, max_depth=6
[17:34:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 2 pruned nodes, max_depth=6
[17:34:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=6
[17:34:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[17:34:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[17:34:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 0 pruned nodes, max_

[17:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=6
[17:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=6
[17:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[17:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=6
[17:34:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 0 pruned nodes, max_depth=6
[17:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=6
[17:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=6
[17:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 2 pruned nodes, max_depth=6
[17:34:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_

[17:35:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=6
[17:35:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=6
[17:35:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[17:35:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=6
[17:35:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=6
[17:35:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 0 pruned nodes, max_depth=6
[17:35:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=6
[17:35:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[17:35:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, ma

/home/shahil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


predictions = [round(value) for value in y_pred_C]
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred_C)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [36]:
#Event_Probability_C and DataFrame  creation
event_prob_C = np.amax(y_prob_C, axis=1)

d_C= {"application_key":X_keyC_lead.astype(int), "y_pred" : y_pred_C,"event_prob" : event_prob_C}
#print X_keyC_lead, y_pred_C, event_prob_C
df_C = pd.DataFrame(data=d_C)

In [37]:
y_prob_C

array([[0.9354288 , 0.06457121],
       [0.8012867 , 0.19871332],
       [0.56797826, 0.43202177],
       ...,
       [0.39623094, 0.60376906],
       [0.84032845, 0.15967152],
       [0.9841498 , 0.01585016]], dtype=float32)

In [38]:
y_pred_C

array([0., 0., 0., ..., 1., 0., 0.])

In [39]:
#Concatenating C and L
frames = [df_C, df_L]
Final_result = pd.concat(frames)
Final_result

,application_key,event_prob,y_pred
0,350054,0.935429,0.0
1,350055,0.801287,0.0
2,350056,0.567978,0.0
3,350062,0.589951,0.0
4,350063,0.909570,0.0
5,350064,0.777881,0.0
6,350065,0.853704,1.0
7,350066,0.616355,1.0
8,350067,0.687950,0.0
9,350068,0.727958,0.0


In [40]:
Final_result.to_csv("dfghj.csv")